In [7]:
#Imports data from Paaraview .csv spreadsheet.
import numpy as np
import math 
from numpy import genfromtxt
data = genfromtxt('dataa.csv' , delimiter = ',')

#Cuts first row (non-physical quantities) and returns matrix with actual data. 
data = data[1:len(data), :]
print(data)

#Creates array with names for each of the columns in the data file with the underlying quantity being shown (bookkeeping). 
data_order = np.array([['Displacement in x'], ['Displacement in y'], ['Displacement in z'], ['Strain in x'], ['Strain in y'], ['Strain in z'], ['Stress in x'], ['Stress in y'], ['Stress in z'], ['Max shear stress (Tresca)'], ['Max Principal Stress'], ['Median Principal Stress'], ['Min Principal Stress'], ['Von Mises Stress'], ['Temperature'], ['x'], ['y'], ['z'], ['Point ID']])
print(data_order)





[[ 7.79978e-07  8.17356e-07 -9.67228e-07 ...  0.00000e+00  0.00000e+00
   0.00000e+00]
 [ 0.00000e+00  0.00000e+00  0.00000e+00 ...  0.00000e+00 -1.00000e-01
   1.00000e+00]
 [-7.99410e-07  8.33655e-07 -1.06757e-06 ...  0.00000e+00  0.00000e+00
   2.00000e+00]
 ...
 [ 5.03804e-07 -5.48304e-08 -1.57579e-06 ...  1.26197e+01  7.50000e-02
   1.55250e+04]
 [-3.18934e-08  4.85283e-07 -1.57676e-06 ...  2.50847e+00  7.50000e-02
   1.55260e+04]
 [ 5.96551e-07 -8.23705e-08 -1.59377e-06 ...  1.34311e+01  7.50000e-02
   1.55270e+04]]
[['Displacement in x']
 ['Displacement in y']
 ['Displacement in z']
 ['Strain in x']
 ['Strain in y']
 ['Strain in z']
 ['Stress in x']
 ['Stress in y']
 ['Stress in z']
 ['Max shear stress (Tresca)']
 ['Max Principal Stress']
 ['Median Principal Stress']
 ['Min Principal Stress']
 ['Von Mises Stress']
 ['Temperature']
 ['x']
 ['y']
 ['z']
 ['Point ID']]


In [8]:
### Performs the post processing by volume averaging method 

# Height of each interface in each model 
interfaces_thin = [-0.1, 0, 0.15, 0.175, 0.325, 0.375, 0.395, 0.4125, 0.4325, 0.450, 0.47]
interfaces_thick = [-0.1, 0, 0.15, 0.175, 0.325, 0.375, 0.395, 0.4125, 0.4825, 0.5, 0.52]
stresses = []
## Helper function that assigns weights to each point
# P_i is the point to which a weight is being assigned 
# P_0 is the reference point for that iteration
# Both arguments have to be a row vector taken from the data matrix 
def weight(P_i, P_0):
    distance = math.sqrt(((P_i[15]-P_0[15])**2)+((P_i[16]-P_0[16])**2)+((P_i[17]-P_0[17])**2))
    if distance!=0:
        W = 1/distance # Simple inverse relation for points around P_0
        return W
    else:
        W = 0 # P_0 is not considered in post-processing itself in this method
        return W

## Iterates for every data point 
data_ave = data

for i in range(len(data)):
    
    # Determines the layer at which the point is
    if interfaces_thin[0]<=data[i, 17]<interfaces_thin[1]:
        layer = 0
    elif interfaces_thin[1]<=data[i, 17]<interfaces_thin[2]:
        layer = 1
    elif interfaces_thin[2]<=data[i, 17]<interfaces_thin[3]:
        layer = 2
    elif interfaces_thin[3]<=data[i, 17]<interfaces_thin[4]:
        layer = 3
    elif interfaces_thin[4]<=data[i, 17]<interfaces_thin[5]:
        layer = 4
    elif interfaces_thin[5]<=data[i, 17]<interfaces_thin[6]:
        layer = 5
    elif interfaces_thin[6]<=data[i, 17]<interfaces_thin[7]:
        layer = 6
    elif interfaces_thin[7]<=data[i, 17]<interfaces_thin[8]:
        layer = 7
    elif interfaces_thin[8]<=data[i, 17]<interfaces_thin[9]:
        layer = 8
    elif interfaces_thin[9]<=data[i, 17]<interfaces_thin[10]:
        layer = 9
    
    # This block creates the finite volume
    # For now let's assume the box size is 2 mm in x and y and 1 mm in z
    boundaryx = np.array([data[i, 15]-1, data[i, 15]+1])
    boundaryy = np.array([data[i, 16]-1, data[i, 16]+1])
    boundaryz = np.array([data[i, 17]-0.5, data[i, 17]+0.5])
    
    # Makes sure finite volume does not cross material boundaries
    if interfaces_thin[layer]>boundaryz[0]:

        boundaryz[0] = interfaces_thin[layer]
    
    if boundaryz[1]>interfaces_thin[layer+1]:
   
        boundaryz[1] = interfaces_thin[layer+1]
        
    # Identifies the points within the finite volume 
    points_in_volume = []
    for j in range(len(data)-1):
        if boundaryx[0]<data[j, 15]<boundaryx[1] and boundaryy[0]<data[j, 16]<boundaryy[1] and boundaryz[0]<data[j, 17]<boundaryz[1]:
            points_in_volume = np.concatenate((points_in_volume, np.array([j])), axis = 0)
    
    # Applies the statistical algorithm to the selected points
    W = []
    ave = 0
    for j in points_in_volume:
        W_j = weight(data[int(j), :], data[int(i), :])
        ave = ave + (W_j*(data[int(j), 10]))
        W = np.concatenate((W, np.array([W_j])), axis = 0)
            
    if sum(W)!=0:
        ave = ave/sum(W)
    else: 
        ave = data[int(i), 10]
    
    stresses = np.concatenate((stresses, np.array([ave])), axis = 0) 
    
print(stresses)
return stresses

data_ave[:, 10] = stresses
print(data_ave)
return data_ave


[ 5.15725879 30.03895453  5.35177711 ... 29.58282341 31.54935985
 28.60586727]


array([ 5.15725879, 30.03895453,  5.35177711, ..., 29.58282341,
       31.54935985, 28.60586727])

array([False, False, False, ..., False, False, False])

In [27]:
(3.4*4)/12.851

1.0582834020698777